In [1]:
import polars as pl
# download latest data source 
import zipfile
from io import BytesIO
from urllib.request import urlopen
import shutil

In [2]:
df = pl.DataFrame._read_parquet('data/openipf-latest.parquet')

In [31]:
users = df["Name"].unique().to_list()
users = [user for user in users if user == "Taylor Atwood"]
users

['Taylor Atwood']

In [53]:
user_df = df.select(
    pl.col(['Date', 'Name', "TotalKg", "Event", "Best3SquatKg",
"Best3BenchKg",
"Best3DeadliftKg"])
).filter(
    pl.col('Name').is_in(users)
).sort(by="Date").drop_nulls()

user_df.to_dict

<bound method DataFrame.to_dict of shape: (20, 7)
┌────────────┬───────────────┬─────────┬───────┬──────────────┬──────────────┬─────────────────┐
│ Date       ┆ Name          ┆ TotalKg ┆ Event ┆ Best3SquatKg ┆ Best3BenchKg ┆ Best3DeadliftKg │
│ ---        ┆ ---           ┆ ---     ┆ ---   ┆ ---          ┆ ---          ┆ ---             │
│ str        ┆ str           ┆ f64     ┆ str   ┆ f64          ┆ f64          ┆ f64             │
╞════════════╪═══════════════╪═════════╪═══════╪══════════════╪══════════════╪═════════════════╡
│ 2013-09-14 ┆ Taylor Atwood ┆ 635.0   ┆ SBD   ┆ 212.5        ┆ 172.5        ┆ 250.0           │
│ 2014-02-01 ┆ Taylor Atwood ┆ 645.0   ┆ SBD   ┆ 215.0        ┆ 170.0        ┆ 260.0           │
│ 2014-07-17 ┆ Taylor Atwood ┆ 687.5   ┆ SBD   ┆ 235.0        ┆ 180.0        ┆ 272.5           │
│ 2014-12-06 ┆ Taylor Atwood ┆ 700.0   ┆ SBD   ┆ 250.0        ┆ 185.0        ┆ 265.0           │
│ …          ┆ …             ┆ …       ┆ …     ┆ …            ┆ …            

In [61]:
import altair as alt
# unpivot data for graphing
user_df = user_df.melt(
    id_vars= ["Date", "Name", "Event"],
    variable_name="Lift",
    value_vars=["Best3SquatKg", "Best3BenchKg", "Best3DeadliftKg"],
    value_name="Weight"
).to_pandas()

user_chart = alt.Chart(user_df).mark_line(point=True).encode(
    x="Date:T",
    y=" :Q",
    color=alt.Color("Lift:N", legend=alt.Legend(title="SBD")),
    tooltip=["Name", "Date", "Event"]
).properties(
    width=800,
    height=600
)

ModuleNotFoundError: pa.Table requires 'pyarrow' module to be installed

In [4]:
schema = {'Name': pl.Utf8,
 'Sex': pl.Utf8,
 'Event': pl.Utf8,
 'Equipment': pl.Utf8,
 'Age': pl.Float64,
 'AgeClass': pl.Utf8,
 'BirthYearClass': pl.Utf8,
 'Division': pl.Utf8,
 'BodyweightKg': pl.Float64,
 'WeightClassKg': pl.Utf8,
 'Squat1Kg': pl.Float64,
 'Squat2Kg': pl.Float64,
 'Squat3Kg': pl.Float64,
 'Squat4Kg': pl.Float64,
 'Best3SquatKg': pl.Float64,
 'Bench1Kg': pl.Float64,
 'Bench2Kg': pl.Float64,
 'Bench3Kg': pl.Float64,
 'Bench4Kg': pl.Float64,
 'Best3BenchKg': pl.Float64,
 'Deadlift1Kg': pl.Float64,
 'Deadlift2Kg': pl.Float64,
 'Deadlift3Kg': pl.Float64,
 'Deadlift4Kg': pl.Float64,
 'Best3DeadliftKg': pl.Float64,
 'TotalKg': pl.Float64,
 'Place': pl.Utf8,
 'Dots': pl.Float64,
 'Wilks': pl.Float64,
 'Glossbrenner': pl.Float64,
 'Goodlift': pl.Float64,
 'Tested': pl.Utf8,
 'Country': pl.Utf8,
 'State': pl.Utf8,
 'Federation': pl.Utf8,
 'ParentFederation': pl.Utf8,
 'Date': pl.Utf8,
 'MeetCountry': pl.Utf8,
 'MeetState': pl.Utf8,
 'MeetTown': pl.Utf8,
 'MeetName': pl.Utf8}

In [21]:
data=pl.read_csv('data/openipf-2023-03-18/openipf-2023-03-18-16bd0ed1.csv', infer_schema_length=None)

op_data = pl.read_csv('data/openpowerlifting-2023-03-18/openpowerlifting-2023-03-18-16bd0ed1.csv', infer_schema_length=None)

In [22]:
op_data.write_parquet('data/openpowerlifting-2023-03-18/openpowerlifting-2023-03-18-16bd0ed1.parquet')

In [23]:
op_data = pl.read_parquet('data/openpowerlifting-2023-03-18/openpowerlifting-2023-03-18-16bd0ed1.parquet')

In [39]:
op_cols = [
    "Name",
    "Sex",
    "Event",
    "Date",
    "MeetCountry",
    "MeetState",
    "Equipment",
    "Best3SquatKg",
    "Best3BenchKg",
    "Best3DeadliftKg", 
    "TotalKg",
    "Wilks",
    "Tested",
    "Federation"
]

In [41]:
filtered_data = op_data.filter(pl.col('Name') == "Nam Tonthat").select(op_cols)

In [43]:
filtered_data

Name,Sex,Event,Date,MeetCountry,MeetState,Equipment,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,TotalKg,Wilks,Tested,Federation
str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,str
"""Nam Tonthat""","""M""","""SBD""","""2022-09-17""","""Australia""",null,"""Raw""",140.0,105.0,200.0,445.0,333.84,"""Yes""","""USAPL"""
"""Nam Tonthat""","""M""","""SBD""","""2022-12-03""","""Australia""","""VIC""","""Raw""",147.5,107.5,207.5,462.5,360.48,"""Yes""","""USAPL"""
"""Nam Tonthat""","""M""","""SBD""","""2021-04-11""","""Australia""","""VIC""","""Raw""",140.0,103.0,190.0,433.0,334.25,"""Yes""","""PA"""
